In [3]:
#  Loading Required Libraries and Data 

import pandas as pd
from IPython.display import display

# file path
file_path = "../data/Common Data Warehouse-Orginal.xlsx"


xls = pd.ExcelFile(file_path)


customer_df = xls.parse("Customer")  # Contains Rating Score and Customer ID
financial_df = xls.parse("Financial")  # Contains Loan Transactions

#  Extract Numeric Rating from Rating Score

def extract_rating_category(score):
    """Extracting the numeric rating from the Rating Score format (PC4+, PP4-, etc.)."""
    score = str(score)  
    numeric_part = ''.join(filter(str.isdigit, score))  

    if numeric_part:
        return int(numeric_part)  
    else:
        return None  

# Applying the function to create a new Rating Category column
customer_df["Rating Category"] = customer_df["Rating Score"].apply(extract_rating_category)

#  Merging Customer Data with Financial Data 

customer_financial_df = financial_df.merge(customer_df, left_on="Cust ID", right_on="Customer ID", how="left")


# Counting the number of loans per customer
loan_counts = customer_financial_df.groupby("Customer ID").agg(
    Loan_Count=("Agmt ID", "count"),
    Rating_Category=("Rating Category", "first")  
)

low_rating_threshold = [0, 1, 2]

# Filtering customers who have more than one loan and have a low rating
multiple_loans_low_rating = loan_counts[(loan_counts["Loan_Count"] > 1) & (loan_counts["Rating_Category"].isin(low_rating_threshold))]

#   Results
print("🔹 Customers with Multiple Loans but Low Ratings:")
display(multiple_loans_low_rating)

#  Save Results to Excel 
output_file_path = "../data/Customers_With_Multiple_Loans_Low_Rating.xlsx"
multiple_loans_low_rating.to_excel(output_file_path, index=False)

print(f"Results saved successfully to: {output_file_path}")


🔹 Customers with Multiple Loans but Low Ratings:


,Loan_Count,Rating_Category
Customer ID,,
1000000280120,3,0.0
1000001743739,3,2.0
1000001785215,4,2.0
1000004417455,12,0.0
1000007671131,16,1.0
...,...,...
1000047843257,5,2.0
1000048391170,6,0.0
1000048392882,3,2.0


Results saved successfully to: ../data/Customers_With_Multiple_Loans_Low_Rating.xlsx
